In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('BostonHousing.csv')
df.head()

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,b,lstat,medv
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33,36.2


In [3]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')

In [4]:
X = df.drop(['medv'],axis=1)
y = df['medv']
scl = StandardScaler()
columns = X.columns
X = scl.fit_transform(X)
X = pd.DataFrame(X, columns=columns)
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.30)
print('X_train shape',X_train.shape)
print('y_train shape',y_train.shape)
print('X_test shape',X_test.shape)
print('y_test shape',y_test.shape)

X_train shape (354, 13)
y_train shape (354,)
X_test shape (152, 13)
y_test shape (152,)


In [5]:
import torch
from torch import nn

In [24]:
# Data class

class data(torch.utils.data.Dataset):
    
    def __init__(self, X, y):
        self.X = torch.from_numpy(X)
        self.y = torch.from_numpy(y)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, i):
        return self.X[i], self.y[i]

class NeuralNetwork(nn.Module):
    
    def __init__(self):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(13, 32),
            nn.Sigmoid(),
            nn.Linear(32, 1)        
        )
    
    def forward(self, inputs):
        return self.layers(inputs)

NN = NeuralNetwork()
train_dataset = data(X_train.values, y_train.values)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=8)


optimizer = torch.optim.SGD(NN.parameters(), lr=1e-4)
cost = nn.MSELoss()
epochs = 20

for epoch in range(epochs):
    print('epoch : ', epoch)
    total_training_loss=0
    for i, data in enumerate(train_loader, 0):
        inputs, targets = data
        inputs, targets = inputs.float(), targets.float()
        targets = targets.reshape((-1, 1))
        optimizer.zero_grad()
        preds = NN.forward(inputs)   # Forward Propagation
        loss = cost(preds, targets)   
        total_training_loss+=loss.item()

        loss.backward()
        optimizer.step()
    print('total training loss :', total_training_loss)


epoch :  0
total training loss : 24143.442169189453
epoch :  1
total training loss : 20999.656997680664
epoch :  2
total training loss : 18301.34637451172
epoch :  3
total training loss : 15969.970031738281
epoch :  4
total training loss : 13945.993713378906
epoch :  5
total training loss : 12183.547149658203
epoch :  6
total training loss : 10646.743858337402
epoch :  7
total training loss : 9307.008628845215
epoch :  8
total training loss : 8141.064582824707
epoch :  9
total training loss : 7129.395711898804
epoch :  10
total training loss : 6255.100748062134
epoch :  11
total training loss : 5503.0848751068115
epoch :  12
total training loss : 4859.547895431519
epoch :  13
total training loss : 4311.701215744019
epoch :  14
total training loss : 3847.653009414673
epoch :  15
total training loss : 3456.3930089473724
epoch :  16
total training loss : 3127.824635744095
epoch :  17
total training loss : 2852.8043928146362
epoch :  18
total training loss : 2623.160792708397
epoch :  19
t

In [30]:
preds = NN.forward(torch.from_numpy(X_test.values).float())

In [31]:
preds = preds.detach().numpy().reshape(-1,)

res = pd.DataFrame( )
res['Actual'] = y_test.values
res['Preds'] = preds
res

,Actual,Preds
0,50.0,16.645601
1,11.9,11.594042
2,15.2,16.054379
3,31.7,22.897537
4,24.7,22.537436
...,...,...
147,33.8,22.937252
148,19.1,14.810785
149,23.8,20.786152
150,8.5,11.925388
